In [2]:
# Import necessary libraries
import pandas as pd
import numpy as np
import io
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display


In [3]:
# Upload dataset
from google.colab import files
uploaded = files.upload()


Saving global_warming_dataset.csv to global_warming_dataset.csv


In [4]:
gw_df = pd.read_csv(io.BytesIO(uploaded['global_warming_dataset.csv']))
print(gw_df.head())

       Country  Year  Temperature_Anomaly  CO2_Emissions    Population  \
0  Country_103  1913            -1.163537   8.876061e+08  1.627978e+08   
1  Country_180  1950            -0.432122   4.497517e+08  4.281359e+08   
2   Country_93  2014             0.444954   4.579080e+08  4.926732e+08   
3   Country_15  2020            -1.171616   5.049503e+08  1.252169e+09   
4  Country_107  1964            -0.564038   6.898891e+08  2.932960e+08   

   Forest_Area           GDP  Renewable_Energy_Usage  Methane_Emissions  \
0    54.872178  6.139887e+12               76.710013       8.317626e+06   
1    84.051006  2.601447e+12               68.450021       6.206540e+06   
2    72.295357  5.192677e+12               36.725699       1.056885e+06   
3    17.259684  8.252128e+12               77.547901       1.986813e+06   
4    44.438605  8.560746e+12               10.019576       3.313252e+06   

   Sea_Level_Rise  ...  Waste_Management  Per_Capita_Emissions  \
0        8.111839  ...         82.6914

In [5]:
# Data Cleaning: Drop duplicates and handle missing values
gw_df = gw_df.drop_duplicates().dropna()


In [6]:
# Dropdown to select country
countries = gw_df['Country'].unique()
country_dropdown = widgets.Dropdown(
    options=['All'] + list(countries),
    value='All',
    description='Country:',
    disabled=False,
)
display(country_dropdown)

Dropdown(description='Country:', options=('All', 'Country_103', 'Country_180', 'Country_93', 'Country_15', 'Co…

In [7]:
def filter_data(country):
    if country != 'All':
        return gw_df[gw_df['Country'] == country]
    return gw_df

CORRELATION ANALYSIS (Sklearn)

In [8]:
def update_plot(country):
    df_filtered = filter_data(country)

    # Correlation analysis
    correlation_features = [
        'Average_Temperature', 'Air_Pollution_Index', 'Sea_Level_Rise', 'Temperature_Anomaly',
        'Industrial_Activity', 'CO2_Emissions', 'Methane_Emissions', 'Fossil_Fuel_Usage'
    ]
    correlation_matrix = df_filtered[correlation_features].corr()

    plt.figure(figsize=(10, 6))
    sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f")
    plt.title(f'Feature Correlation with Global Warming Indicators - {country}')
    plt.show()

    # Anomaly detection
    iso_forest = IsolationForest(n_estimators=100, contamination=0.05, random_state=42)
    outlier_predictions = iso_forest.fit_predict(df_filtered[['CO2_Emissions', 'Temperature_Anomaly']])
    df_filtered['Outlier'] = outlier_predictions

    plt.figure(figsize=(10, 6))
    sns.scatterplot(x=df_filtered['CO2_Emissions'], y=df_filtered['Temperature_Anomaly'], hue=df_filtered['Outlier'],
                    palette={1: 'blue', -1: 'red'})
    plt.xlabel("CO2 Emissions")
    plt.ylabel("Temperature Anomaly")
    plt.title(f"Anomaly Detection in CO2 Emissions & Weather Patterns - {country}")
    plt.show()

    # Population Growth vs Greenhouse Gas Emissions
    plt.figure(figsize=(10, 6))
    sns.regplot(x=df_filtered['Population'], y=df_filtered['CO2_Emissions'], scatter_kws={'alpha':0.5})
    plt.xlabel("Population Growth")
    plt.ylabel("CO2 Emissions")
    plt.title(f"Population Growth Influence on CO2 Emissions - {country}")
    plt.show()

widgets.interactive(update_plot, country=country_dropdown)


interactive(children=(Dropdown(description='Country:', options=('All', 'Country_103', 'Country_180', 'Country_…